In [1]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

In [2]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}
            
    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping
            

        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)
             
        return X_encoded 

In [3]:
syn_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043_set9.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.190512,0.190512,-0.322003,0.190512,-1.069622,-0.322003,-0.322003,-1.069622,0.000000,0.190512,0.000000,-0.322003,-1.069622,0.190512,0.613831,-1.069622,0.190512,-0.322003,0.000000,-1.069622
1,0.190512,0.190512,0.613831,0.613831,1.001670,0.613831,-0.322003,0.613831,-0.322003,0.190512,0.190512,1.001670,0.613831,0.190512,0.190512,-0.322003,0.613831,0.190512,0.190512,-1.069622
2,0.190512,0.613831,-1.069622,0.000000,-1.069622,-1.069622,0.000000,-1.069622,-1.069622,0.613831,-0.322003,-1.069622,1.001670,0.190512,0.000000,-1.069622,0.000000,0.190512,0.000000,0.613831
3,0.190512,-1.069622,0.190512,-1.069622,0.190512,0.190512,-1.069622,0.190512,0.190512,-0.322003,0.190512,-1.069622,-0.322003,-1.069622,1.001670,0.190512,-0.322003,0.190512,-1.069622,0.613831
4,0.190512,-0.322003,-1.069622,0.613831,0.613831,0.190512,0.190512,-0.322003,-0.322003,0.613831,-1.069622,0.190512,-1.069622,1.001670,-1.069622,0.613831,0.613831,-1.069622,0.000000,0.613831


In [4]:
org_test_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.569715,-0.155969,1.586662,-0.110356,-0.021324,-0.412361,0.229132,-0.033252,-0.033252,-0.482741,0.699000,0.610469,0.630512,0.728991,0.268615,0.367391,0.733324,0.331370,0.885785,0.967358
1,-1.172108,-0.155969,-1.191184,-1.455578,0.023052,0.318783,0.229132,0.003738,0.169368,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,0.733324,0.331370,-0.692514,-0.506478
2,0.967358,-0.155969,-1.233862,-0.259956,-0.021324,-0.412361,-0.696401,0.003738,-0.141946,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-2.375949,-0.650735,-0.422533,0.569715
3,0.136166,-0.155969,1.586662,0.967358,-0.021324,0.318783,0.229132,0.003738,-0.141946,0.722046,0.699000,0.610469,-0.269859,0.728991,0.268615,0.367391,0.733324,-0.650735,0.885785,-0.390795
4,0.136166,-0.155969,-1.593819,1.363492,0.023052,0.318783,0.229132,0.003738,-0.141946,-0.482741,0.699000,0.610469,0.630512,-0.831894,0.268615,0.367391,-2.375949,-0.650735,-0.692514,0.136166


In [5]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1370 169 185 389
pod:  0.6777003484320557
pof:  0.10981156595191682
AUC:  0.7839443912400695
recall:  0.6777003484320557
precision:  0.6971326164874552
F1-Score:  0.6872791519434629
accuracy:  0.8324656885944155


In [6]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1348 191 172 402
pod:  0.7003484320557491
pof:  0.12410656270305392
AUC:  0.7881209346763476
recall:  0.7003484320557491
precision:  0.6779089376053963
F1-Score:  0.6889460154241647
accuracy:  0.8282063416942735


In [7]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1391 148 253 321
pod:  0.5592334494773519
pof:  0.09616634178037686
AUC:  0.7315335538484875
recall:  0.5592334494773519
precision:  0.6844349680170576
F1-Score:  0.6155321188878236
accuracy:  0.8102224325603408


In [8]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1418 121 271 303
pod:  0.5278745644599303
pof:  0.07862248213125406
AUC:  0.7246260411643382
recall:  0.5278745644599303
precision:  0.714622641509434
F1-Score:  0.6072144288577154
accuracy:  0.8144817794604827


In [9]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1420 119 384 190
pod:  0.3310104529616725
pof:  0.07732293697205977
AUC:  0.6268437579948063
recall:  0.3310104529616725
precision:  0.6148867313915858
F1-Score:  0.4303510758776897
accuracy:  0.7619498343587316


In [10]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1434 105 309 265
pod:  0.4616724738675958
pof:  0.0682261208576998
AUC:  0.6967231765049481
recall:  0.4616724738675958
precision:  0.7162162162162162
F1-Score:  0.5614406779661016
accuracy:  0.804070042593469


In [11]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0) 

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1458 81 320 254
pod:  0.4425087108013937
pof:  0.05263157894736842
AUC:  0.6949385659270126
recall:  0.4425087108013937
precision:  0.7582089552238805
F1-Score:  0.5588558855885588
accuracy:  0.8102224325603408


In [12]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix
 
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)    

Epoch 1/30
124/124 [==============================] - 2s 7ms/step - loss: 0.2635 - accuracy: 0.8968 - val_loss: 0.2322 - val_accuracy: 0.8976
Epoch 2/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1905 - accuracy: 0.9252 - val_loss: 0.1945 - val_accuracy: 0.9158
Epoch 3/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1680 - accuracy: 0.9333 - val_loss: 0.1835 - val_accuracy: 0.9280
Epoch 4/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1579 - accuracy: 0.9386 - val_loss: 0.1751 - val_accuracy: 0.9290
Epoch 5/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1509 - accuracy: 0.9412 - val_loss: 0.1747 - val_accuracy: 0.9320
Epoch 6/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1410 - accuracy: 0.9462 - val_loss: 0.1779 - val_accuracy: 0.9391
Epoch 7/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1371 - accuracy: 0.9483 - val_loss: 0.1748 - val_accuracy: 0.9351
Epoch 